In [3]:
from surprise import Dataset
from surprise import Reader
from surprise.dataset import BUILTIN_DATASETS #с помощью данного объекта мы можем использовать встроенные датасеты

data = Dataset.load_from_file(
    "data/u.data.txt",
    reader=Reader(line_format="user item rating timestamp", sep="\t"),
)

In [4]:
import pandas as pd
df = pd.DataFrame(data.raw_ratings, columns=['userId', 'movieId', 'rating', 'timestamp'])

In [5]:
df.apply('nunique')

userId         943
movieId       1682
rating           5
timestamp    49282
dtype: int64

In [6]:
df['rating'].value_counts()

rating
4.0    34174
3.0    27145
5.0    21201
2.0    11370
1.0     6110
Name: count, dtype: int64

In [7]:
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.25, random_state=13)
len(testset)

25000

In [8]:
from surprise import SVD, KNNBasic, accuracy

In [9]:
sim_options = {
    'name': 'cosine',
    'user_based': False
}
 
knn = KNNBasic(sim_options=sim_options)

In [10]:
knn.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [11]:
predictions = knn.test(testset)
predictions

[Prediction(uid='7', iid='633', r_ui=5.0, est=4.199452349030111, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='422', iid='287', r_ui=3.0, est=3.4703437660463736, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='804', iid='163', r_ui=3.0, est=3.5716736533692854, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='189', iid='480', r_ui=5.0, est=4.222825780855538, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='238', iid='546', r_ui=3.0, est=3.473417286928204, details={'actual_k': 17, 'was_impossible': False}),
 Prediction(uid='804', iid='216', r_ui=4.0, est=3.922551907749182, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='350', iid='204', r_ui=4.0, est=4.345238219480267, details={'actual_k': 38, 'was_impossible': False}),
 Prediction(uid='708', iid='993', r_ui=4.0, est=3.4458505791534115, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='193', iid='1078', r_ui=4.0, es

In [12]:
for prediction in predictions:
    if prediction.uid == '500' and prediction.iid == '699':
        print(prediction.r_ui)
        print(round(prediction.est, 2))
        break

3.0
3.47


In [13]:
uid = str(500)
iid = str(699)  
pred = knn.predict(uid, iid, verbose=True)

user: 500        item: 699        r_ui = None   est = 3.47   {'actual_k': 40, 'was_impossible': False}


In [14]:
accuracy.rmse(predictions)

RMSE: 1.0272


1.0271678039029761

In [15]:
pred = pd.DataFrame(predictions)
pred.sort_values(by=['est'],inplace=True,ascending = False)

In [16]:
recom = pred[pred.uid =='849']['iid'].to_list()

In [17]:
sim_options = {
    'name': 'cosine',
    'user_based': True
}
knn = KNNBasic(sim_options=sim_options)
knn.fit(trainset)
predictions = knn.test(testset)
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0175


1.0174852296380237

In [18]:
model = SVD()
model.fit(trainset)
predictions = model.test(testset)
accuracy.rmse(predictions)

RMSE: 0.9403


0.9402935548700366